In [ ]:
# default_exp model

# Model

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import torch

import pytorch_lightning as pl

from torch import nn, Tensor
from torch.nn import CrossEntropyLoss, TransformerEncoder, TransformerEncoderLayer
from transformers import AdamW, AutoConfig, AutoModelForCausalLM, get_cosine_with_hard_restarts_schedule_with_warmup as cosine_restart_schedule
from transformers.models.encoder_decoder.modeling_encoder_decoder import shift_tokens_right

In [ ]:
# export
class RetroEncoder(nn.Module):
    def __init__(self, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.pos_emb = nn.Parameter(torch.zeros(1, d_model, d_hid))
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.d_model = d_model

        self.drop = nn.Dropout(dropout)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        pos_embeddings = self.pos_emb[:, :x.shape[1], :]
        x = self.drop(x + pos_embeddings)
        output = self.transformer_encoder(x, mask)
        return output

In [ ]:
# some tests for the above encoder
encoder = RetroEncoder(d_model=768, nhead=8, d_hid=768, nlayers=6)
test_data = torch.randn(1, 10, 768)
# test_mask = torch.ones(1, 10)
output = encoder(test_data)
assert output.shape == (1, 10, 768)

In [ ]:
# export
class RetroFitModel(nn.Module):
    def __init__(self, encoder_model: nn.Module, decoder_model_name: str):
        super().__init__()
        self.encoder_model = encoder_model

        decoder_config = AutoConfig.from_pretrained(decoder_model_name)
        decoder_config.is_decoder = True
        decoder_config.add_cross_attention = True
        self.decoder_model = AutoModelForCausalLM.from_pretrained(decoder_model_name, config=decoder_config)
    
    def forward(self, x: Tensor, mask: Tensor, labels: Tensor) -> Tensor:
        encoder_output = self.encoder_model(x, mask)
        decoder_input_ids = shift_tokens_right(
            labels, self.decoder_model.config.eos_token_id, self.decoder_model.config.bos_token_id
        )
        decoder_output = self.decoder_model(
            input_ids=decoder_input_ids, encoder_hidden_states=encoder_output
        )
        logits = decoder_output.logits
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.reshape(-1, self.decoder_model.config.vocab_size), labels.view(-1))

        return loss

In [ ]:
# some tests for the above model
retro_model = RetroFitModel(encoder_model=encoder, decoder_model_name='gpt2')
test_data = torch.randn(1, 10, 768)
test_mask = torch.ones(1, 10)
test_labels = torch.randint(0, retro_model.decoder_model.config.vocab_size, (1, 10))
loss = retro_model(test_data, None, test_labels)
assert loss.shape == ()
assert loss.dtype == torch.float32
assert retro_model.decoder_model.config.vocab_size == 50257

In [ ]:
# export
# Code from: https://github.com/huggingface/transformers/blob/master/examples/research_projects/codeparrot/scripts/codeparrot_training.py#L113
def get_grouped_params(model, weight_decay, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [ ]:
weight_decayed_params, non_weight_decayed_params = get_grouped_params(retro_model, weight_decay=0.01)

assert len(weight_decayed_params["params"]) > 0
assert len(non_weight_decayed_params["params"]) > 0

In [ ]:
# export
class RetroFitModelWrapper(pl.LightningModule):
    def __init__(
        self,
        model,
        weight_decay=0.1,
        lr=5e-4,
        num_warmup_steps=2_000,
        freeze_decoder=True
    ):
        super().__init__()
        self.model = model
        self.weight_decay = weight_decay
        self.lr = lr
        self.num_warmup_steps = num_warmup_steps
        self.freeze_decoder = freeze_decoder

        self.save_hyperparameters()

    def training_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["retrieved_input_ids"], labels=batch["input_ids"]).loss

        self.log("trn_loss", loss, on_step=True, on_epoch=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["retrieved_input_ids"], labels=batch["input_ids"]).loss

        self.log("val_loss", loss, on_epoch=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        # Prepare the optimizer and learning rate scheduler
        param_model = self.model if not self.freeze_decoder else self.model.encoder
        optimizer = AdamW(get_grouped_params(param_model, self.weight_decay), lr=self.lr)
        lr_scheduler = cosine_restart_schedule(
            optimizer=optimizer,
            num_warmup_steps=self.num_warmup_steps,
            num_training_steps=self.total_steps(),
        )
        return [optimizer], [lr_scheduler]
    
    def total_steps(self) -> int:
        """The number of total training steps that will be run. Used for lr scheduler purposes."""
        dataset_size = len(self.trainer.datamodule.train_dataloader())
        num_devices = max(1, self.trainer.gpus)  # TODO: consider num_tpu_cores
        effective_batch_size = self.trainer.datamodule.batch_size * num_devices
        return (dataset_size / effective_batch_size) * self.trainer.max_epochs

    def generate(self, retrieved_examples, encoder_tokenizer, decoder_tokenizer, skip_special_tokens=True):
        retrieved = [encoder_tokenizer.eos_token.join(r) for r in retrieved_examples]
        input_ids = encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        generated = self.model.generate(input_ids, decoder_start_token_id=self.model.config.decoder.bos_token_id)

        return decoder_tokenizer.decode(generated, skip_special_tokens=skip_special_tokens)

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()